# Merge Flights with Corresponding Weather Information

## Weather data
Weather data was obtained from the Iowa Environmental Mesonet (IEM) website. A python script was used to download data by setting limits on the start and end date as well as specifying the stations we intended to use based on our selected airports of interest.

The following stations were taken into consideration:

* NY (New York)
* NC (North Carolina)
* TX (Texas)
* WA (Washington)
* CA (California)
* IL (Illinios)
* ID (Idaho)
* IA (Iowa)

### Weather Data Description

* *station*:three or four character site identifier
* *valid*:timestamp of the observation
* *tmpf*:Air Temperature in Fahrenheit, typically @ 2 meters
* *dwpf*:Dew Point Temperature in Fahrenheit, typically @ 2 meters
* *relh*:Relative Humidity in %
* *drct*:Wind Direction in degrees from north
* *sknt*:Wind Speed in knots
* *p01i*:One hour precipitation for the period from the observation time to the time of the previous hourly precipitation reset. This varies slightly by site. Values are in inches. This value may or may not contain frozen precipitation melted by some device on the sensor or estimated by some other means. Unfortunately, we do not know of an authoritative database denoting which station has which sensor.
* *alti*:Pressure altimeter in inches
* *mslp*:Sea Level Pressure in millibar
* *vsby*:Visibility in miles
* *gust*:Wind Gust in knots
* *skyc1*:Sky Level 1 Coverage
* *skyc2*:Sky Level 2 Coverage
* *skyc3*:Sky Level 3 Coverage
* *skyc4*:Sky Level 4 Coverage
* *skyl1*:Sky Level 1 Altitude in feet
* *skyl2*:Sky Level 2 Altitude in feet
* *skyl3*:Sky Level 3 Altitude in feet
* *skyl4*:Sky Level 4 Altitude in feet
* *wxcodes*:Present Weather Codes (space seperated)
* *feel*:Apparent Temperature (Wind Chill or Heat Index) in Fahrenheit
* *ice_accretion_1hr*:Ice Accretion over 1 Hour (inches)
* *ice_accretion_3hr*:Ice Accretion over 3 Hours (inches)
* *ice_accretion_6hr*:Ice Accretion over 6 Hours (inches)
* *peak_wind_gust*:Peak Wind Gust (from PK WND METAR remark) (knots)
* *peak_wind_drct*:Peak Wind Gust Direction (from PK WND METAR remark) (deg)
* *peak_wind_time*:Peak Wind Gust Time (from PK WND METAR remark)
* *metar*:unprocessed reported observation in METAR format

## Airport Geolocation
Airport geolocation data was obtained from the following website: https://openflights.org.

### Airport Data Description


 * *Airport ID:* Unique OpenFlights identifier for this airport.
 * *Name:* Name of airport. May or may not contain the City name.
 * *City:* Main city served by airport. May be spelled differently from Name.
 * *Country:* Country or territory where airport is located. See countries.dat to cross-reference to ISO 3166-1 codes.
 * *IATA:* 3-letter IATA code. Null if not assigned/unknown.
 * *ICAO:* 4-letter ICAO code.
 * *Null:* if not assigned.
 * *Latitude:* Decimal degrees, usually to six significant digits. Negative is South, positive is North.
 * *Longitude:* Decimal degrees, usually to six significant digits. Negative is West, positive is East.
 * *Altitude:* In feet.
 * *Timezone:* Hours offset from UTC. Fractional hours are expressed as decimals, eg. India is 5.5.
 * *DST:* Daylight savings time. One of E (Europe), A (US/Canada), S (South America), O (Australia), Z (New Zealand), N (None) or U (Unknown). See also: Help: Time
 * *Tz:* database time zone	Timezone in "tz" (Olson) format, eg. "America/Los_Angeles".
 * *Type:* Type of the airport. Value "airport" for air terminals, "station" for train stations, "port" for ferry terminals and "unknown" if not known. In airports.csv, only type=airport is included.
 * *Source:*	Source of this data. "OurAirports" for data sourced from OurAirports, "Legacy" for old data not matched to OurAirports (mostly DAFIF), "User" for unverified user contributions. In airports.csv, only source=OurAirports is included.


## Merging Weather and Flight Data

### Preprocessing Weather Data

Weather data was given by station name and geolocation however it was not linked to any nearby airport. Also, the data was given at 20 minute intervals. Therefore it was neccessary to preprocess the data. The following steps were completed:

1. The time was converted to epoch time in milliseconds rounded to the base hour. 
2. After rounding the time to the base hour, each station had multiple similar entries, so only the first one was kept and the rest were dropped.
3. Creating a merg ID that was a combination of the epoch time and the station name (epochTime_stationID).

### Preprocessing Flights Data

1. Flights with missing timestamps (year, month, day and time) were dropped
2. Airports not of interest were dropped limmiting the data to the following airports:
    * DFW
    * ORD
    * DEN
    * LAX
    * LGA
    * RDU
    * DCA
    * BOS
    * SAN
    * SJC
3. The time was converted to epoch time in milliseconds rounded to the base hour.
4. Finding the nearest weather station to each airport
5. Creating a merg ID that was a combination of the epoch time and the station name (epochTime_stationID).

### Merging Weather and Flight Data

Data was merged using pandas library function "merge" based on merge ID using a left join strategy. 



In [1]:
import pandas as pd
import numpy as np
import math
import geopy.distance
import matplotlib.pyplot as plt

import time, sys
from IPython.display import clear_output
from scipy.spatial.distance import cdist
from datetime import datetime

## Global Variables

In [27]:
print("reading flights csv...")
flights = pd.read_csv('flights_2008_processed.csv')
display(flights.shape)
print("reading airports csv...")
airport_locations = pd.read_csv('airport_locations.csv')

print("reading weather csv...")
weather_data = pd.read_csv('MERGED_STA_SAMPLE.csv')

print('Done')

reading flights csv...


/home/rms/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1580558, 31)

reading airports csv...
reading weather csv...
Done


## Function Definitions

In [18]:
# find the nearest airport to each station and add it to weather_data['IATA']
def closest_point(point, points, airport_iata):
    """ Find closest point from a list of points. """
    x = cdist([point], points).argmin()
    return airport_iata[x]

## Data processing

In [19]:
print('selecting data of interest only...')
airports_interest = ["DFW","ORD","DEN","LAX","LGA","RDU","DCA","BOS","SAN","SJC"]
airport_locations = airport_locations[airport_locations['IATA'].isin(airports_interest)]
print('Done')

selecting data of interest only...
Done


In [20]:
print("merging airport_locations lat,lon...")
airport_locations['point'] = [(x, y) for x,y in zip(airport_locations['Latitude'], airport_locations['Longitude'])]
print('Done')

merging airport_locations lat,lon...
Done


In [21]:
print("merging weather_data lat,lon...")
## doing this here again because for some reason cdist function is complaining
## when it is done previously and read from the csv file
weather_data['point'] = [(x, y) for x,y in zip(weather_data['lat'], weather_data['lon'])]
print('Done')

merging weather_data lat,lon...
Done


In [22]:
print('find the nearest station to each airport...')
weather_copy = weather_data.drop_duplicates('station')
airport_station = {}
for station, point in zip(airport_locations.IATA.unique(),airport_locations.point.unique()):
    airport_station[station] = closest_point(point, list(weather_copy['point']), weather_copy['station'].tolist()) 
print('Done')

find the nearest station to each airport...
Done


In [23]:
flights['nearest_weather_sta'] = [airport_station.get(station, '') for station in flights['Origin']]
print('Done')

Done


In [24]:
print('creating merge_id column...')
flights['merge_id'] = ["{}_{}".format(epoch, iata) for epoch, iata in zip(flights['epoch_time'], flights['nearest_weather_sta'])]
print('Done')

creating merge_id column...
Done


## Merge Weather Dataframes

In [25]:
print('merging weather data into flights...')
flights = pd.merge(flights, weather_data, how='left', on='merge_id')
print('Done')

merging weather data into flights...
Done


In [ ]:
print ('writing file to csv...')
flights.to_csv(path_or_buf='./flights_weather_merged.csv', sep=',')
print('Done')

In [28]:
display(flights.shape)
display(flights.head(2))

(1580558, 31)

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,epoch_time
0,320,2008,1,3,4,1735.0,1710,2010.0,1955,WN,...,10.0,0,NaN,0,0.0,0.0,4.0,0.0,11.0,1.199380e+12
1,321,2008,1,3,4,2105.0,2025,2342.0,2310,WN,...,9.0,0,NaN,0,0.0,0.0,11.0,0.0,21.0,1.199394e+12
